In [4]:
import requests
from PIL import Image
from fastapi import FastAPI
from fastapi.responses import HTMLResponse
import gradio as gr

SERVER_URL = "http://127.0.0.1:5000/process"



In [5]:
import paramiko
import socket
import select

def create_ssh_tunnel(server_ip, server_port, username, private_key_path, local_port, remote_port):
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server_ip, port=server_port, username=username, key_filename=private_key_path)

    transport = client.get_transport()
    local_sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    local_sock.bind(('localhost', local_port))
    local_sock.listen(1)

    print(f"Forwarding local port {local_port} to {remote_port} on {server_ip}...")
    while True:
        conn, addr = local_sock.accept()
        channel = transport.open_channel(
            'direct-tcpip',
            (server_ip, remote_port),
            conn.getsockname()
        )

        while True:
            r, w, x = select.select([conn, channel], [], [])
            if conn in r:
                data = conn.recv(1024)
                if not data:
                    break
                channel.send(data)
            if channel in r:
                data = channel.recv(1024)
                if not data:
                    break
                conn.send(data)
        conn.close()
        channel.close()



In [ ]:
# Настройте параметры
create_ssh_tunnel(
    server_ip='89.169.132.198',
    server_port=22,
    username='yrsolo',
    private_key_path='C:\\Users\\solofarm\\.ssh\\id_ed25519',
    local_port=9001,
    remote_port=9001
)


Forwarding local port 9001 to 9001 on 89.169.132.198...


In [ ]:
yc certificate-manager certificate get-content  --id fpqca9ot3om9d5a68rj1 --output-certificate ~/tg-det/https-sert/certificate.pem  --output-private-key ~/tg-det/https-sert/private_key.pem

In [ ]:
yc certificate-manager certificate get-content --id fpqca9ot3om9d5a68rj1  --chain  --output-cert ~/tg-det/https-sert/certificate.pem  --output-key ~/tg-det/https-sert/private_key.pem


In [ ]:
yc certificate-manager certificate content  --id fpqca9ot3om9d5a68rj1  --chain ~/tg-det/https-cert/certificate.pem --key ~/tg-det/https-cert/private_key.pem --key-format pkcs8

In [ ]:
openssl x509 -in ~/tg-det/https-cert/certificate.pem -text -noout

In [ ]:
yc iam key create --service-account-id aje1kqd422vq2vefkbbl  --output ~/tg-det/https-cert/cert-updater-key.json